# Environment

In [106]:
from __future__ import print_function, division
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile

from scipy.stats import bernoulli
from scipy.stats import beta as bt
from scipy.stats import dirichlet
from scipy.special import digamma
from scipy.stats import loggamma 
from scipy.stats import expon
from scipy.cluster.vq import whiten

import time

import random

import wave
from sklearn.decomposition import FastICA

In [8]:
data_paths = [["C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset1\\rsm2_mB.wav", 
        "C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset1\\rsm2_mA.wav"],
       ["C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset2\\rss_mB.wav",
       "C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset2\\rss_mA.wav"],
       ["C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset3\\rssd_B.wav",
       "C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset3\\rssd_A.wav"]]


data_paths = np.array(data_paths)

# Functions

## Audio Write Functions

In [182]:
from IPython.display import display, Audio

def write_to_audio(w, x, rates):
    for i in range(w.shape[0]):
        out = w[i,:].dot(x)
        out = out.astype(np.int16)
        st = "out%d.wav" % i
        scipy.io.wavfile.write(st, rates[i], out)
        display(Audio(filename=st))

## Negentropy Functions

In [11]:
def g_1(u):
    a_1 = 1
    u_cosh = np.cosh(a_1 * u)
    u_log = np.log(u_cosh)
    final = (1/a_1) * u_log
    return final
def g_1_derivative(u):
    a_1 = 1
    return np.tanh(a_1 * u)

In [12]:
def g_2(u):
    return -1 * np.exp(-1 * (u * u) / 2)
def g_2_derivative(u):
    return u * np.exp(-1 * (u * u) / 2)

## Update Functions

## Hyperparameters

In [193]:
sample_size = 10000
n = 2
d = 2
threshold = 0.01

## Data Loading

In [194]:
def center(z):
    return z / np.mean(z)

In [217]:
current_sample = 0
rates = []
data = []
for i in range(n):
    rate, dat = scipy.io.wavfile.read(data_paths[current_sample,i])
    rates.append(rate)
    data.append(dat)

## Variables

In [228]:
t = data[0].shape[0]
x = np.zeros((n, t))
                                              
for i in range(n):
    x[i,:] = data[i]
print(x)
#x = whiten(x)

# whiten x

eigvalues, eigvectors = np.linalg.eig(np.cov(x))
x = eigvectors.dot(np.diag(np.sqrt(eigvalues))).dot(eigvectors.T).dot(x)

# center x
for i in range(n):
    x[i,:] = center(x[i,:])
    

w = np.random.rand(n, d)
for i in range(n):
    w[i,:] = w[i,:] / np.linalg.norm(w[i,:])

print(w)

[[ 2732.  2868.  2564. ..., -1198. -2648. -2960.]
 [ 3812.  5380.  4391. ..., -1756. -1939. -3867.]]
[[ 0.98798731  0.15453501]
 [ 0.92560213  0.37849796]]


In [229]:
print(x)

[[-6252.27306321 -7650.770811   -6509.48076045 ...,  2808.25416752
   4674.93417164  6566.48051936]
 [-7069.56089646 -9507.75259289 -7866.10946026 ...,  3227.82656804
   4194.41400534  7261.24550463]]


# Iterative Loop

In [230]:
w_record = []
dot_product_record = []
loss_record = []

In [231]:
def w_has_not_converged(w, w_old, threshold):
    print(w, w_old)
    s = w.dot(w_old)
    s_1 = np.absolute(1-s)
    if s_1 < threshold:
        return False
    return True

for i in range(n):
    j=1
    w_old = np.zeros(w.shape)
    while(w_has_not_converged(w[i,:], w_old[i,:], threshold)):
        if j % 10 == 0:
            print("Iteration %d is starting" % j)
            print(w)
            
        w_old = np.copy(w)
        
            
        # Update

        g_exp = w[i,:].dot(x)
        g_exp = g_2(g_exp)
        g_exp = g_exp[np.newaxis,:].T
        g_exp = x.dot(g_exp)
        g_exp = g_exp / x.shape[1]
        g_exp = g_exp.T.squeeze()
        
        
        g_der_exp = w[i,:].dot(x)
        g_der_exp = g_2_derivative(g_der_exp)
        g_der_exp = g_der_exp.dot(np.ones([int(x.shape[1]), 1]))
        g_der_exp = g_der_exp / x.shape[1]
        g_der_exp = g_der_exp * w[i,:]
        
        w[i,:] = g_exp - g_der_exp
        
        print("wi")
        print(w[i,:])
        
        # Decorrelate
        
        
        
        total = np.zeros(w[i,:].shape)
        for k in range(i):
            temp = w[i,:].dot(w[k,:])
            total = total + temp * w[k,:]
        w[i,:] = w[i,:] - total
        
        
        # Normalize
        
        w[i,:] = w[i,:] / np.linalg.norm(w[i,:])
        
        # Update j
        
        j = j + 1


[ 0.98798731  0.15453501] [ 0.  0.]
wi
[-0.00160127  0.01015994]
[-0.15568496  0.98780676] [ 0.98798731  0.15453501]
wi
[-0.02120689 -0.0033556 ]
[-0.98771164 -0.1562873 ] [-0.15568496  0.98780676]
wi
[ 0.00252941 -0.01590689]
[ 0.15704065 -0.98759214] [-0.98771164 -0.1562873 ]
wi
[-0.09121954 -0.01448232]
[-0.98763045 -0.15679956] [ 0.15704065 -0.98759214]
wi
[ 0.00164521 -0.01020079]
[ 0.15922527 -0.98724228] [-0.98763045 -0.15679956]
wi
[-0.00466852 -0.00075164]
[-0.98728592 -0.15895444] [ 0.15922527 -0.98724228]
wi
[-0.00400861  0.02481192]
[-0.15949171  0.98719927] [-0.98728592 -0.15895444]
wi
[  3.28878536e-04   6.92326447e-05]
[ 0.97855275  0.2059964 ] [-0.15949171  0.98719927]
wi
[ 0.0086771 -0.041433 ]
[ 0.20497802 -0.97876658] [ 0.97855275  0.2059964 ]
Iteration 10 is starting
[[ 0.20497802 -0.97876658]
 [ 0.92560213  0.37849796]]
wi
[ 0.14129275  0.02948345]
[ 0.97891471  0.20426939] [ 0.20497802 -0.97876658]
wi
[ 0.00327052 -0.01532409]
[ 0.20872288 -0.97797483] [ 0.9789147

KeyboardInterrupt: 

# Evaluation

In [183]:
w = np.array([[-0.70986198, -0.7043408 ]
 ,[ 0.89442719,  0.4472136 ]])
write_to_audio(w, x, rates)

# Ground Truth Using sklearn

In [ ]:
from IPython.display import display, Audio

transformer = FastICA(n_components=2)

y = np.zeros((n, t), dtype=np.int16)
rate, y[0,:] = scipy.io.wavfile.read(data_paths[1,0])
rate, y[1,:] = scipy.io.wavfile.read(data_paths[1,1])

x_transformed = transformer.fit_transform(y.astype(np.float64) )


z = np.zeros((n, t), dtype=np.int16)

nt = np.array([0, 0.7043408 ])
z[0,:] = x_transformed[0,:].dot(y)
z[1,:] = x_transformed[1,:].dot(y)
z = z.astype(np.int16)

#y = y.astype(int16)
scipy.io.wavfile.write("testout0.wav", rate, z[0,:])
scipy.io.wavfile.write("testout1.wav", rate, z[1,:])

display(Audio(filename="testout0.wav"))
display(Audio(filename="testout1.wav"))


# Misc

In [ ]:
plt.figure(figsize=[8,6], dpi=150)
plt.plot(range(len(dot_product_record)), dot_product_record)
plt.ylabel("Signal")
plt.xlabel("Time")
plt.title("Signal Over Time")
plt.show()

In [ ]:
print(dot_product_record[:100])